In [1]:
from google.colab import files
files.upload() #upload kaggle.json

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# !kaggle kernels list — user YOUR_USER — sort-by dateRun

# !kaggle kernels output shahules/zomato-complete-eda-and-lstm-model -p .

# !unzip -q train.csv.zip -d .
# !unzip -q test.csv.zip -d .
!ls

Saving kaggle.json to kaggle.json
kaggle.json
kaggle.json  sample_data


In [2]:
!kaggle datasets download --force bryanpark/sudoku

 72% 49.0M/68.1M [00:00<00:00, 78.8MB/s]
100% 68.1M/68.1M [00:00<00:00, 93.4MB/s]


In [3]:
!unzip sudoku.zip 

Archive:  sudoku.zip
  inflating: sudoku.csv              


In [4]:
import tensorflow as tf

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
try:
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
  # Invalid device or cannot modify virtual devices once initialized.
  pass

In [ ]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

def get_model():

    model = keras.models.Sequential()

    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
    model.add(BatchNormalization())
    model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

    model.add(Flatten())
    model.add(Dense(81*9))
    model.add(Reshape((-1, 9)))
    model.add(Activation('softmax'))
    
    return model

In [6]:
import copy
import keras
import pandas as pd
import numpy as np
# from model import get_model
# from scripts.data_preprocess import get_data

In [ ]:
!unzip sudoku.zip

Archive:  sudoku.zip
replace sudoku.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
df=pd.read_csv('sudoku.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   quizzes    1000000 non-null  object
 1   solutions  1000000 non-null  object
dtypes: object(2)
memory usage: 15.3+ MB


In [9]:
features=df['quizzes']
labels=df['solutions']

In [10]:
feat=[]
for i in features:
    x=np.array([int(j) for j in i]).reshape((9,9,1))
    feat.append(x)

In [11]:
feat=np.array(feat)

In [12]:
feat=feat/9

In [13]:
feat -=.5

In [14]:
    labe=[]
    for i in labels:
    
        x = np.array([int(j) for j in i]).reshape((81,1)) - 1
        labe.append(x)  

In [15]:
label = np.array(labe)

In [16]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(feat,label,test_size=0.3,random_state=42)

In [17]:
import keras
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape

model = keras.models.Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=(9,9,1)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))
model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

In [18]:
import tensorflow as tf

In [19]:
adam = tf.keras.optimizers.Adam(lr=.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)



/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [20]:
model.fit(x_train,y_train,batch_size=32,epochs=2)

Epoch 1/2
21875/21875 [==============================] - 99s 4ms/step - loss: 0.4550
Epoch 2/2
21875/21875 [==============================] - 87s 4ms/step - loss: 0.3610


In [21]:
def norm(a):
    
    return (a/9)-.5

In [22]:
def denorm(a):
    
    return (a+.5)*9

In [23]:
def inference_sudoku(sample):
    
    '''
        This function solve the sudoku by filling blank positions one by one.
    '''
    
    feat = copy.copy(sample)
    
    while(1):
    
        out = model.predict(feat.reshape((1,9,9,1)))  
        out = out.squeeze()

        pred = np.argmax(out, axis=1).reshape((9,9))+1 
        prob = np.around(np.max(out, axis=1).reshape((9,9)), 2) 
        
        feat = denorm(feat).reshape((9,9))
        mask = (feat==0)
     
        if(mask.sum()==0):
            break
            
        prob_new = prob*mask
    
        ind = np.argmax(prob_new)
        x, y = (ind//9), (ind%9)

        val = pred[x][y]
        feat[x][y] = val
        feat = norm(feat)
    
    return pred

In [24]:
def test_accuracy(feats, labels):
    
    correct = 0
    
    for i,feat in enumerate(feats):
        
        pred = inference_sudoku(feat)
        
        true = labels[i].reshape((9,9))+1
        
        if(abs(true - pred).sum()==0):
            correct += 1
        
    print(correct/feats.shape[0])

In [25]:
test_accuracy(x_test[:100], y_test[:100])

1.0


In [26]:
def solve_sudoku(game):
    
    game = game.replace('\n', '')
    game = game.replace(' ', '')
    game = np.array([int(j) for j in game]).reshape((9,9,1))
    game = norm(game)
    game = inference_sudoku(game)
    return game

In [28]:
game = '''
         3 0 6 5 0 8 4 0 0 
         5 2 0 0 0 0 0 0 0 
         0 8 7 0 0 0 0 3 1 
         0 0 3 0 1 0 0 8 0
         9 0 0 8 6 3 0 0 5 
         0 5 0 0 9 0 6 0 0 
         1 3 0 0 0 0 2 5 0 
         0 0 0 0 0 0 0 7 4 
         0 0 5 2 0 6 3 0 0 
      '''

game = solve_sudoku(game)

print('solved puzzle:\n')
print(game)

solved puzzle:

[[3 1 6 5 7 8 4 9 2]
 [5 2 9 1 3 4 8 6 7]
 [4 8 7 6 2 9 5 3 1]
 [6 7 3 4 1 5 9 8 9]
 [9 4 1 8 6 3 7 2 5]
 [8 5 2 7 9 2 6 4 3]
 [1 3 4 9 8 7 2 5 6]
 [2 6 8 3 5 1 9 7 4]
 [7 9 5 2 4 6 3 1 8]]


In [29]:
np.sum(game, axis=1)

array([45, 45, 45, 52, 45, 46, 45, 45, 45])